In [2]:
QRY_PATH="/home/global/anaconda2/tract_querier/queries/freesurfer_queries.qry"
#SUBJ_ID="T02S01"
SUBJ_ID="p_AR28_na"
NAS="tupac"
#protocole="healthy_volunteers"
protocole="alexcis"
DIR_PATH="/NAS/"+NAS+"/protocoles/"+protocole
FS_PATH=DIR_PATH+"/FS53/"
DTI_PATH=FS_PATH+SUBJ_ID+"/dti/"
TRACTO="whole_brain_6_1500000"
#SEG = DTI_PATH+"raparc+aseg.nii"
SEG = DTI_PATH+"raparc.a2009s+aseg.nii"
MESH = "lh.white.ras"

In [5]:
from nibabel import trackvis as tv
from dipy.viz import fvtk
from dipy.segment.metric import ResampleFeature
from dipy.segment.metric import AveragePointwiseEuclideanMetric
from dipy.segment.metric import CenterOfMassFeature
from dipy.segment.metric import EuclideanMetric
from dipy.segment.metric import IdentityFeature
from dipy.segment.clustering import QuickBundles
from dipy.io.image import load_nifti, save_nifti
from dipy.tracking.streamline import set_number_of_points
from nibabel import freesurfer as fs
from dipy.utils.optpkg import optional_package
import dipy.io.vtk as io_vtk
import dipy.viz.utils as ut_vtk
from dipy.viz import window, actor, ui
from dipy.tracking import utils
import nipype.interfaces.mrtrix as mrt
from dipy.tracking.streamline import select_random_set_of_streamlines
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib

In [8]:
SEG2 = DTI_PATH+"raparc.a2009s+aseg.nii"
nii2 = nib.load(SEG2)

SEG1 = DTI_PATH+"aparc.a2009s+aseg.nii"
nii1 = nib.load(SEG1)

#data_seg, affine = load_nifti(SEG)
#print(type(nii2.ImageArrayProxy))
#print(type(data_seg))
data_colorfa = nib.load(DTI_PATH+"dti_finalcor_FA.nii.gz")
data_dti=data_colorfa.get_data();
plt.imshow(data_dti[64,:,:])
plt.colorbar()
#plt.show()

raparc = fvtk.slicer(nii2.get_data(),nii2.affine)
raparc.display(40, None, None)
data_seg=nii1.get_data();
white_matter = ((data_seg == 251) | (data_seg == 252) | (data_seg == 253) | (data_seg == 254) | (data_seg == 255)) 
#plt.imshow(white_matter[126,:,:])
#plt.show()

aparc = fvtk.slicer(white_matter,nii1.affine)
aparc.display(126, None, None)

#tractogram=DTI_PATH+"whole_brain_6_1500000_part000099.tck"
#tck = nib.streamlines.load(tractogram)
#print(tck.header)

tractogram=DTI_PATH+"part1__front_middle.trk"
trk = nib.streamlines.load(tractogram)

bundle_native = transform_streamlines(trk.tractogram.streamlines,np.linalg.inv(data_colorfa.affine))
stream_actor2 = actor.line(bundle_native,data_dti, linewidth=0.1)
qb = QuickBundles(threshold=10.)
clusters = qb.cluster(bundle_native)

print("hjjkk")
print("Nb. clusters:", len(clusters))
print("Cluster sizes:", map(len, clusters))
print("Small clusters:", clusters < 10)
print("Streamlines indices of the first cluster:\n", clusters[0].indices)
print("Centroid of the last cluster:\n", clusters[-1].centroid)

bar = actor.scalar_bar()

r = fvtk.ren()
#fvtk.add(r, raparc)
fvtk.add(r, aparc)
fvtk.add(r, bar)
fvtk.add(r,stream_actor2)
fvtk.add(r, fvtk.streamtube(trk.tractogram.streamlines,fvtk.colors.blue,linewidth=0.1,opacity=0.3));
#fvtk.add(r, fvtk.streamtube(tck.tractogram.streamlines,fvtk.colors.white,linewidth=0.1,opacity=0.3));
fvtk.show(r)

FileNotFoundError: No such file: '/NAS/tupac/protocoles/alexcis/FS53/p_AR28_na/dti/raparc.a2009s+aseg.nii'

In [22]:
SEG = DTI_PATH+"raparc.a2009s+aseg.nii"

TRACTO_TCK = DTI_PATH+"whole_brain_6_1500000_part0000*.tck"
TRACTO_TRK = DTI_PATH+"part1.tck"
OUTPUT = DTI_PATH+"part1"+"_"

!tckedit $TRACTO_TCK $TRACTO_TRK
!python /home/romain/Téléchargements/tck2trk.py -f $SEG $TRACTO_TRK
!rm $TRACTO_TRK
TRACTO_TRK = DTI_PATH+"part1.trk"
!tract_querier -a $SEG -t $TRACTO_TRK -q "/home/global/anaconda2/tract_querier/queries/myFirstQuery.qry" -o $OUTPUT
!rm $TRACTO_TRK

TRACTO_TCK = DTI_PATH+"whole_brain_6_1500000_part0001*.tck"
TRACTO_TRK = DTI_PATH+"part2.tck"
OUTPUT = DTI_PATH+"part2"+"_"

!tckedit $TRACTO_TCK $TRACTO_TRK
!python /home/romain/Téléchargements/tck2trk.py -f $SEG $TRACTO_TRK
!rm $TRACTO_TRK

TRACTO_TRK = DTI_PATH+"part2.trk"
!tract_querier -a $SEG -t part2.trk -q "/home/global/anaconda2/tract_querier/queries/myFirstQuery.qry" -o $OUTPUT
!rm $TRACTO_TRK


Loading files
Traceback (most recent call last):
  File "/home/global/anaconda2/bin/tract_querier", line 286, in <module>
    main()
  File "/home/global/anaconda2/bin/tract_querier", line 141, in main
    options.tractography_file_name
  File "/home/global/anaconda2/lib/python2.7/site-packages/tract_querier/tractography/__init__.py", line 46, in tractography_from_file
    return tractography_from_trackvis_file(filename)
  File "/home/global/anaconda2/lib/python2.7/site-packages/tract_querier/tractography/trackvis.py", line 78, in tractography_from_trackvis_file
    tracts_and_data, header = trackvis.read(filename, points_space='rasmm')
  File "/home/global/anaconda2/lib/python2.7/site-packages/nibabel/trackvis.py", line 153, in read
    fileobj = ImageOpener(fileobj)
  File "/home/global/anaconda2/lib/python2.7/site-packages/nibabel/openers.py", line 156, in __init__
    self.fobj = opener(fileish, *args, **kwargs)
IOError: [Errno 2] No such file or directory: 'part2.trk'


In [95]:
import numpy as np
from dipy.viz import window, actor
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
from dipy.tracking.streamline import transform_streamlines

fetch_bundles_2_subjects()
dix = read_bundles_2_subjects(subj_id='subj_1', metrics=['fa'],bundles=['cg.left', 'cst.right'])
affine = dix['affine']
print(affine)

Data size is approximately 234MB
Dataset is already in place. If you want to fetch it again please first remove the folder /home/romain/.dipy/exp_bundles_and_maps 
[[   1.            0.           -0.         -120.55383301]
 [   0.            1.           -0.         -104.46057129]
 [  -0.           -0.            1.          -28.35990906]
 [   0.            0.            0.            1.        ]]


In [38]:
SEG = DTI_PATH+"raparc.a2009s+aseg.nii"
!tract_querier -a "/NAS/tupac/protocoles/alexcis/FS53/p_AR28_na/mri/wmparc.nii.gz" -t $DTI_PATH"whole_brain_6_1500000.trk" -q $QRY_PATH -o $DTI_PATH"bundle"

Loading files
Calculating labels and crossings
Computing queries
	Query af.left: 000498
	Query af.right: 000233
	Query cb.left: 018344
	Query cb.right: 019478
	Query cc: 000000
	Query cc_1: 001102
	Query cc_2: 031498
	Query cc_3: 004313
	Query cc_4: 009851
	Query cc_5: 000000
	Query cc_6: 000000
	Query cc_7: 012441
	Query cst.left: 000956
	Query cst.right: 004141
	Query ec.left: 000135
	Query ec.right: 000444
	Query extreme_capsule.left: 002362
	Query extreme_capsule.right: 001099
	Query ilf.left: 003866
	Query ilf.right: 003927
	Query internal_capsule_new.left: 000899
	Query internal_capsule_new.right: 001398
	Query ioff.left: 000707
	Query ioff.right: 001046
	Query mdlf.left: 002249
	Query mdlf.right: 001409
	Query slf_i.left: 003128
	Query slf_i.right: 001703
	Query slf_ii.left: 005496
	Query slf_ii.right: 005453
	Query slf_iii.left: 001213
	Query slf_iii.right: 002161
	Query soff.left: 000008
	Query soff.right: 000005
	Query striato_frontal.left: 008529
	Query striato_frontal.right

In [4]:
#!tckconvert -force $DTI_PATH"whole_brain_6_1500000.tck" $DTI_PATH"whole_brain_6_1500000.vtk"
#!mkdir $DTI_PATH"bundle"
!echo tract_querier -a $SEG -t $DTI_PATH"whole_brain_6_1500000.vtk" -q $QRY_PATH -o $DTI_PATH"bundle"
#!tract_querier -a $SEG -t $DTI_PATH"whole_brain_6_1500000.vtk" -q $QRY_PATH -o $DTI_PATH"bundle"

tract_querier -a /NAS/tupac/protocoles/alexcis/FS53/p_AR28_na/dti/raparc.a2009s+aseg.nii -t /NAS/tupac/protocoles/alexcis/FS53/p_AR28_na/dti/whole_brain_6_1500000.vtk -q /home/global/anaconda2/tract_querier/queries/freesurfer_queries.qry -o /NAS/tupac/protocoles/alexcis/FS53/p_AR28_na/dti/bundle


In [9]:
#Lecture de la surface
vtk, have_vtk, setup_module = optional_package('vtk')
white_mesh=fs.read_geometry(FS_PATH+SUBJ_ID+"/surf/"+MESH)
my_polydata = vtk.vtkPolyData()
ut_vtk.set_polydata_vertices(my_polydata, white_mesh[0])
ut_vtk.set_polydata_triangles(my_polydata, white_mesh[1])

#lecture des images de parcelation
data_seg, affine = load_nifti(SEG)
vol_actor = fvtk.slicer(data_seg,affine)
vol_actor.display(40, None, None)
print(affine)

streams, hdr = tv.read(DTI_PATH+TRACTO+".trk")
input_spacetmp=np.array(tv.aff_from_hdr(hdr));
input_space=np.array([[1.,0.,0.,-input_spacetmp[0,3]],[0.,1.,0.,input_spacetmp[1,3]],[0.,0.,1.,input_spacetmp[2,3]],[0.,0.,0.,1.]])
whole_brain = [i[0] for i in streams]
whole_brain_red = select_random_set_of_streamlines(whole_brain, 1000)
whole_brain_red=list(utils.move_streamlines(whole_brain_red,input_space))

[[  -2.            0.            0.          128.34214783]
 [   0.            2.            0.         -129.92842102]
 [   0.            0.            2.          -58.10770035]
 [   0.            0.            0.            1.        ]]


/mnt/software/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:15: FutureWarning: Defaulting to `atleast_v2` of False.  Future versions will default to True
  from ipykernel import kernelapp as app


In [6]:
import nibabel as nib
from nibabel.affines import apply_affine

r = fvtk.ren()
fvtk.add(r, vol_actor)
stream_actor = ut_vtk.get_actor_from_polydata(my_polydata)
stream_actor.GetProperty().SetOpacity(.6) 
r.add(stream_actor)

fvtk.add(r, fvtk.streamtube(whole_brain_red,fvtk.colors.white,linewidth=0.1,opacity=0.3));
li = ['fs_uf.right','fs_uf.left'] 
for BUNDLE in li:
    streams, hdr = tv.read("/home/romain/Téléchargements/wmql_example_dataset/results/"+BUNDLE+".trk")
    tracto = [i[0] for i in streams]
    feature = ResampleFeature(nb_points=240)
    metric = AveragePointwiseEuclideanMetric(feature=feature)  # a.k.a. MDF
    qb = QuickBundles(threshold=100, metric=metric)
    qb.max_nb_clusters = 1
    cluster = qb.cluster(tracto)
    
    plot_tracto = select_random_set_of_streamlines(tracto, 1000)
    plot_tracto=list(utils.move_streamlines(plot_tracto,input_space))
    clus=list(utils.move_streamlines(cluster.centroids,input_space))
    
    fvtk.add(r, fvtk.streamtube(plot_tracto,fvtk.colors.blue,linewidth=0.1,opacity=0.3));
    fvtk.add(r, fvtk.streamtube(clus,linewidth=1));
    
    
a=fvtk.axes()
fvtk.add(r,a)

fvtk.show(r)

NameError: name 'vol_actor' is not defined

In [5]:
from dipy.align.streamlinear import StreamlineLinearRegistration
from dipy.align.streamlinear import distance_matrix_mdf
from dipy.tracking.streamline import set_number_of_points
from dipy.tracking.streamline import center_streamlines

cb_subj1 = set_number_of_points(clus[0], 20)
cb_subj1=np.asarray(cb_subj1)
srr = StreamlineLinearRegistration()

test1 = set_number_of_points(clus, 100)
test2 = set_number_of_points(whole_brain_red, 100)
test3 = set_number_of_points(plot_tracto, 100)

DM1=distance_matrix_mdf(test1,test2)
print(DM1)

DM2=distance_matrix_mdf(test1,test3)
print(type(DM2>8))
print(type(plot_tracto))
itertools.compress(plot_tracto,DM2>8)
#for fib in tracto:
#    cb_subj2 = set_number_of_points(fib, 20)
#    srm = srr.optimize(static=cb_subj1,moving=cb_subj2)
#srm = srr.optimize(static=test1,moving=test2)

[[  67.90398463   61.9023146    90.63839649   64.19068362   88.4149526
    71.95366031   53.13097031   61.6456306    24.47149989   47.92549395
    53.7686795    36.69803713   50.68625539   39.8807327    69.24040963
    58.27722314   55.90524908   47.34260617   78.51046625   72.58647789
    50.23474565   30.44066727   85.002666     62.12799183   24.93264532
    42.39237037   35.1240071    60.82734129   80.9921213    49.30779139
    64.63813954   81.85516986   69.7603411    75.17541998   48.36718399
    66.98532406   45.33452824   24.25552238   93.58983304   41.97159517
    66.32741473   70.31837372   59.80591161   50.32487615   84.92888485
    65.83284866   62.76618155   87.18137232   65.35204761   65.25617266
    50.27040662   49.38586446   47.22903382   78.21597513   32.75095144
    40.22152831   81.87242123   44.17735974   18.37315069   51.35877975
    98.44160871   29.89785596   64.29504762   97.22378069   67.72857718
    94.46466356   51.9227384    49.8216443   101.66851688   30.93

NameError: name 'itertools' is not defined

In [6]:
import numpy as np
import dipy.tracking.learning as tl
from scipy.spatial import distance
#arr = tl.detect_corresponding_tracks([0], clus, tracto)
test=np.asarray(clus[0])
print(test.shape)
print(test.size)
print(type(test))

print(fib.shape)
print(fib.size)
print(type(fib))

distance.cdist(set_number_of_points(test, 5),set_number_of_points(fib,5),'cityblock')

(240, 3)
720
<type 'numpy.ndarray'>


NameError: name 'fib' is not defined